In [ ]:


# important imports
import numpy as np
import math
import itertools
import time
from IPython.display import clear_output
import csv
import ast
import collections
import random
import matplotlib.pyplot as plt
import networkx as nx
import collections
from timeit import timeit
import pickle


In [ ]:
#Given an omino (n1xn2 binary matrix), checks if it is valid
# i.e. all ones are rook-adjacent and contiguous

def valid_omino(omino):
    '''
    takes an omino as input and validates it
    returns a boolean
    
    works by checking that all of the 1s in the omino
    are in a single connected component
    '''
    if 2 in omino: return False
    n1 = omino.shape[0]
    n2 = omino.shape[1]
    check = np.zeros([n1,n2]) - omino
    
    
    _f  = False
    for i in range(n1):
        for j in range(n1):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    # I know this should require fewer passes, but it's fast enough anyway...
    for iter in range(n1*n2):
        for i in range(n1):
            for j in range(n2):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2

                    if i != n1-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != n2-1:
                        check[i,j+1] = check[i,j+1]**2
                    
    return np.sum(check) == np.sum(omino)




def make_omino(inds, dim):
    '''
    takes a set of indices (x,y) to be set to 1
    makes a matrix of size dim x dim
    returns the matrix if it is a valid omino
    returns the empty zeros matrix otherwise
    
    
    '''
    p = np.zeros([dim[0],dim[1]])
    for loc in inds:
        p[loc[0],loc[1]] = 1

    if valid_omino(p):
        return p.astype(int)
    else:
        return np.zeros([0,0])
    
    
    
    
# generates all of the <cells>-ominos in the <grid>^2 grid
def make_omino_set(cells, grid):
    '''
    takes a number and a pair, cells and grid and returns a list
    of all the valid cells-ominos which fit in the
    grid[0]*grid[1] grid
    
    works by checking exhaustive enumeration of all
    ominos with #cells 1s
    
    ** note -- for 5-ominos and larger, it's probably faster
       to call make_omino_set() for a smaller set and use
       biggen() to enlarge them
    
    returns a list of valid ominos    
    '''
    
    if len(grid) !=2:
        print("make_omino_set() requires two dimensions!")
    
    
    
    pair_idx = []
    for i in range(grid[0]):
        for j in range(grid[1]):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
                
        p = make_omino(t,grid).astype(int)
        
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos

In [ ]:
# the queen contiguity versions of the above functions
# checks if an omino is queen-contiguity valid
def valid_omino_qc(omino):
    n1 = omino.shape[0]
    n2 = omino.shape[1]
    check = np.zeros([n1,n2]) - omino
    
    
    _f  = False
    for i in range(n1):
        for j in range(n2):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    for iter in range(n1*n2):
        for i in range(n1):
            for j in range(n2):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2


                    if i != n1-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != n2-1:
                        check[i,j+1] = check[i,j+1]**2
                        
                    if i!= 0 and j!=0:
                        check[i-1,j-1] = check[i-1,j-1]**2
                        
                    if i!=0 and j!= n2-1:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i != n1-1 and j != 0:
                        check[i+1,j-1] = check[i+1,j-1]**2
                   
                    if j != n1-1 and i != 0:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i!= n1-1 and j != n2-1:
                        check[i+1,j+1] = check[i+1,j+1]**2
                    
                    
    return np.sum(check) == np.sum(omino)




def make_omino_qc(inds, dim):

    p = np.zeros([dim[0],dim[1]])
    for loc in inds:
        p[loc[0],loc[1]] = 1

    if valid_omino(p):
        return p.astype(int)
    else:
        return np.zeros([0,0])
    
    
    
    
# generates all of the <cells>-ominos in the <grid>^2 grid
def make_omino_set_qc(cells, grid):

    
    if len(grid) !=2:
        print("make_omino_set() requires two dimensions!")
    
    
    
    pair_idx = []
    for i in range(grid[0]):
        for j in range(grid[1]):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
                
        p = make_omino_qc(t,grid).astype(int)
        
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos

In [ ]:
def biggen(om_list, grid, bad, accp):
    '''
        takes a list of ominos, the grid size, and
        a list of holes which are "bad", and returns
        the list of ominos which can be formed by
        changing a zero to a one in any of the 
        ominos in the list

        ** bad and accp don't do anything right now -- a future
        version will use it to smartly check for ominos
        which may be valid, but cannot be part of a 
        valid plan
        
        works by checking the validity of every enlargement
        of each omino in om_list, and keeping only ones it 
        hasn't seen before
    '''
    
    
    new_oms = []
    count = 0.0
    step = np.sum(om_list[0])+1
    for o in om_list:
        count+=1
        for i in range(grid[0]):
            for j in range(grid[1]):
                zeros = np.zeros([grid[0],grid[1]])
                zeros[i,j]+=1

                if o[i,j] == 0 and valid_omino(zeros+o):
                    if len(new_oms)==0: new_oms.append(zeros+o)
                    else:
                        seen =  False
                        for x in new_oms[::-1]:
                            if not seen and 1 not in x-zeros+o:
                                seen = True
                        if not seen:

                            new_oms.append(zeros+o)
    return new_oms



def biggen_qc(om_list, grid, bad, accp):
    ''' 
    the qc version of the above
    '''
    new_oms = []
    count = 0.0
    step = np.sum(om_list[0])+1
    for o in om_list:
        count+=1
        for i in range(grid[0]):
            for j in range(grid[1]):
                zeros = np.zeros([grid[0],grid[1]])
                zeros[i,j]+=1

                if o[i,j] == 0 and valid_omino_qc(zeros+o):
                    if len(new_oms)==0: new_oms.append(zeros+o)
                    else:
                        seen =  False
                        for x in new_oms[::-1]:
                            if not seen and 1 not in x-zeros+o:
                                seen = True
                        if not seen:

                            new_oms.append(zeros+o)
    return new_oms










def check_holes(om, grid, bad_hole_sizes, num_parts, cont='rc'):
    '''
    if cont=='rc': checks that the omino doesn't have any connected components
    that are of a size in bad_hole_sizes -- for example, if you are doing
    a partition of the 4x4 grid into parts of size 4, then you can't fill
    a hole of size 1, so we know that piece can't be part of a valid plan
    also checks that the provided omino doesn't split the grid into
    too many pieces, which is also a condition for invalidity
    
    if cont=='qc': does nothing
    
    
    '''
    
    
    
    if cont == 'qc': return True
    gr = nx.grid_2d_graph(grid[0],grid[1])
    
    
    
    
    for i in range(grid[0]-1):
        for j in range(grid[1]-1):
            if om[i,j] != om[i+1,j]: 
                gr.remove_edge((i,j),(i+1,j))
            if om[i,j] != om[i,j+1]: 
                gr.remove_edge((i,j),(i,j+1))
    for i in range(grid[0]-1):
        if om[i,grid[1]-1] != om[i+1,grid[1]-1]: 
            gr.remove_edge((i,grid[1]-1),(i+1,grid[1]-1))
    for j in range(grid[1]-1):
        if om[grid[0]-1,j] != om[grid[0]-1,j+1]: 
            gr.remove_edge((grid[0]-1,j),(grid[0]-1,j+1))
                
    pts = 0
    for comp in nx.connected_components(gr):
        pts +=1
        if comp in bad_hole_sizes: return False
    
    return pts <= num_parts
    
    



def first_zero_mat(mat):
    ''' returns the index (reading left to right, top to bottom)
        of the first 0 in a matrix.
        
        e.g. if the first row of a matrix reads [1,1,1,0,0],
        then return (0,3)
    '''
    return tuple(np.transpose(np.nonzero(mat == 0))[0].tolist())

def last_one_mat(mat):
    ''' returns the index (reading left to right, top to bottom)
        of the last 1 in a matrix.
    '''
    return tuple(np.transpose(np.nonzero(mat == 1))[-1].tolist())

In [ ]:
def build_conflicts(omlist,bads, num_parts, cont='rc' ):
    '''
        builds two dictionaries:
        
        firstdict has keys as tuples (i,j)
        where an omino in omlist is in the
        set at key (i,j) iff its first zero
        is at index (i,j)
        
        
        conflicts has each omino as a key and
        its values are the list of ominos which
        are pairwise invalid together in a plan
        e.g. omino B will be in omino A's set
        if they overlap or if the composition
        of them creates a hole which can't be filled (if rc)
    
    
    '''

    inds = range(len(omlist))
    done = set()
    firstdict = collections.defaultdict(set)
    for i in range(omlist[0].shape[0]):
        for j in range(omlist[0].shape[1]):
            for ix in inds:
                if ix not in done and omlist[ix][i,j] == 1:
                    done.add(ix)
                    firstdict[(i,j)].add(ix)
    

    conflict = collections.defaultdict(set)
    for i,j in itertools.combinations(inds,2):
        if 2 in omlist[i] + omlist[j]:
            conflict[i].add(j)
            conflict[j].add(i)
        elif not check_holes(omlist[i] + omlist[j], omlist[0].shape, bads, num_parts, cont):
            conflict[i].add(j)
            conflict[j].add(i)
    
    return firstdict, conflict





In [ ]:
def get_next(fd, curr_inds, oms):
    '''
        given a list of ominos, finds the index of the first zero and returns
        the list of ominos which have their first 1 in that position
    '''
    
    
    
    return [ d for d in fd[ first_zero_mat(sum([  oms[j] for o in curr_inds      ]))   ]   if verify(curr_inds)  ]


def get_hole_sizes(om_sizes):
    '''
    creates a non-exhaustive list of hole sizes which
    can't be filled by ominos of sizes in om_sizes
    
    returns the list of sizes strictly smaller than the
    smallest thing in om_sizes and anything between
    the largest thing and twice the smallest thing
    [either of these may be empty]
    
    '''
    L = list(range(1,min(om_sizes)    ))
    L2 = list(range(max(om_sizes)+1, 2*min(om_sizes)))
    
    return L+L2


In [36]:

def enumerator(grid, om_sizes, num_parts, cont='rc' ):
    vp = []
    count = 0
    
    
    #build ominos
    oms = []
    
    bad_hole_sizes = get_hole_sizes(om_sizes)
 
    if cont == 'rc':
        start_oms = make_omino_set(1,grid)




    elif cont == 'qc':
        start_oms = make_omino_set_qc(1,grid)

    if 1 in om_sizes: oms += start_oms
    for i in range(max(om_sizes)-1):

        if cont == 'rc':
            start_oms = biggen(start_oms,grid, bad_hole_sizes, num_parts)
        elif cont == 'qc':
            start_oms = biggen_qc(start_oms,grid, bad_hole_sizes, num_parts)


        if np.sum(start_oms[0]) in om_sizes:
            oms += start_oms

        
        
        
        
        
        
    
    #make conflict
    fd, cf = build_conflicts(oms, bad_hole_sizes, num_parts, cont)
    
    def verify(plan):
        for i,j in itertools.combinations(plan,2):
            if j in cf[i]: return False
        return True


    def recurs_part(plan,steps):
        nonlocal fd, count, oms
        if steps == 0:
            count += (np.sum( sum(    [  oms[j] for j in plan     ])    ) == grid[0]*grid[1])
            return
        if (np.sum( sum(    [  oms[j] for j in plan     ])    ) == grid[0]*grid[1]) and len(plan)<num_parts: return
        for q in fd[ first_zero_mat( sum([ oms[j] for j in plan  ]) ) ]:
            if verify(plan+[q]):
                recurs_part(plan+[q], steps-1)                


    
    
    
    
    
    for p in fd[(0,0)]:

        recurs_part([p],num_parts-1)
    

    return count, vp

In [ ]:
print("RC -2x2 into 2")
tic = time.time()
print(enumerator([2,2],[2],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3,4],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3,4],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],list(range(1,4)),2, cont='rc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("QC -2x2 into 2")
tic = time.time()
print(enumerator([2,2],[2],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3,4],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],[1,2,3,4],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([2,2],list(range(1,4)),2, cont='qc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("RC -3x3 into 3")
tic = time.time()
print(enumerator([3,3],[3],3, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[2,3,4],3, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[1,2,3,4,5],3, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[1,2,3,4,5,6],3, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],list(range(1,8)),3, cont='rc'))
#print(_)
print(time.time()-tic)


In [ ]:
print("QC -3x3 into 3")
tic = time.time()
print(enumerator([3,3],[3],3, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[2,3,4],3, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[1,2,3,4,5],3, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],[1,2,3,4,5,6],3, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([3,3],list(range(1,8)),3, cont='qc'))
#print(_)
print(time.time()-tic)


In [ ]:
print("RC -4x4 into 2")
tic = time.time()
print(enumerator([4,4],[8],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[7,8,9],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[6,7,8,9,10],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[5,6,7,8,9,10,11],2, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],list(range(1,16)),2, cont='rc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("QC -4x4 into 2")
tic = time.time()
print(enumerator([4,4],[8],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[7,8,9],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[6,7,8,9,10],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[5,6,7,8,9,10,11],2, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],list(range(1,16)),2, cont='qc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("RC -4x4 into 4")
tic = time.time()
print(enumerator([4,4],[4],4, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[3,4,5],4, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[2,3,4,5,6],4, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4,5,6,7],4, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],list(range(1,14)),4, cont='rc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("QC -4x4 into 4")
tic = time.time()
print(enumerator([4,4],[4],4, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[3,4,5],4, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[2,3,4,5,6],4, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4,5,6,7],4, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],list(range(1,14)),4, cont='qc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("RC -4x4 into 8")
tic = time.time()
print(enumerator([4,4],[2],8, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3],8, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4],8, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4,5],8, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],list(range(1,10)),8, cont='rc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("qc -4x4 into 8")
tic = time.time()
print(enumerator([4,4],[2],8, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3],8, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4],8, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([4,4],[1,2,3,4,5],8, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
#print(enumerator([4,4],list(range(1,10)),8, cont='qc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("RC -5x5 into 5 , pm3++")
tic = time.time()
print(enumerator([5,5],[5],5, cont='rc', pregen=False))
#print(_)
print(time.time()-tic)
tic = time.time()
#print(enumerator([5,5],[4,5,6],5, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
#print(enumerator([5,5],[3,4,5,6,7],5, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
#print(enumerator([5,5],[2,3,4,5,6,7,8],5, cont='rc', pregen=True))
#print(_)
print(time.time()-tic)
tic = time.time()
#print(enumerator([5,5],list(range(1,22)),5, cont='rc'))
#print(_)
print(time.time()-tic)


In [ ]:
print("QC -5x5 into 5")
tic = time.time()
print(enumerator([5,5],[5],5, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([5,5],[4,5,6],5, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([5,5],[3,4,5,6,7],5, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([5,5],[2,3,4,5,6,7,8],5, cont='qc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([5,5],list(range(1,22)),5, cont='qc'))
#print(_)
print(time.time()-tic)

In [ ]:
print("RC -6x6 into 18")
tic = time.time()
print(enumerator([6,6],[2],18, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([6,6],[1,2,3],18, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([6,6],[1,2,3,4],18, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print(enumerator([6,6],[1,2,3,4,5],18, cont='rc'))
#print(_)
print(time.time()-tic)
tic = time.time()
print("???")
#print(_)
print(time.time()-tic)